In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_losses, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '2', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 1000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.05, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent_topic', 32, 'dim_latent_topic')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

In [8]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

# run model 

## init

In [9]:
def get_feed_dict(batch, mode='train'):
    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    keep_prob = config.keep_prob if mode == 'train' else 1.0
    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [23]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow])
t_variables['keep_prob'] = tf.placeholder(tf.float32)

# tree_idxs = {0:[1, 2], 
#              1:[10, 11], 2:[20, 21, 22], 
#              10: [100, 101], 11: [110, 111, 112], 20: [200, 201], 21: [210, 211], 22:[220, 221, 222]
#              }

# tree_idxs = {0:[1, 2, 3], 
#                       1:[10, 11], 2:[20, 21], 3:[30, 31]}

tree_idxs = {0:[1, 2, 3], 
                      1:[10, 11], 2:[20, 21], 3:[30, 31],
                      10: [100, 101], 11: [110, 111], 20: [200, 201], 21: [210, 211], 30:[300, 301], 31:[310, 311]}


topic_idxs = [0] + [idx for child_idxs in tree_idxs.values() for idx in child_idxs]
child_to_parent_idxs = {child_idx: parent_idx for parent_idx, child_idxs in tree_idxs.items() for child_idx in child_idxs}

## doubly rnn

In [24]:
class DoublyRNNCell:
    def __init__(self, dim_hidden, output_layer=None):
        self.dim_hidden = dim_hidden
        
        self.ancestral_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='ancestral')
        self.fraternal_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='fraternal')
        self.hidden_layer = tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='hidden')
        
        self.output_layer=output_layer
        
    def __call__(self, state_ancestral, state_fraternal, reuse=True):
        with tf.variable_scope('input', reuse=reuse):
            state_ancestral = self.ancestral_layer(state_ancestral)
            state_fraternal = self.fraternal_layer(state_fraternal)

        with tf.variable_scope('output', reuse=reuse):
            state_hidden = self.hidden_layer(state_ancestral + state_fraternal)
            if self.output_layer is not None: 
                output = self.output_layer(state_hidden)
            else:
                output = state_hidden
            
        return output, state_hidden
    
    def get_initial_state(self, name):
        initial_state = tf.get_variable(name, [1, self.dim_hidden], dtype=tf.float32)
        return initial_state
    
    def get_zero_state(self, name):
        zero_state = tf.zeros([1, self.dim_hidden], dtype=tf.float32, name=name)
        return zero_state

In [25]:
def doubly_rnn(dim_hidden, tree_idxs, initial_state_parent=None, initial_state_sibling=None, output_layer=None, name=''):
    outputs, states_parent = {}, {}
    
    with tf.variable_scope(name, reuse=False):
        doubly_rnn_cell = DoublyRNNCell(dim_hidden, output_layer)

        if initial_state_parent is None: initial_state_parent = doubly_rnn_cell.get_initial_state('init_state_parent')
        if initial_state_sibling is None: 
#             initial_state_sibling = doubly_rnn_cell.get_initial_state('init_state_sibling')
            initial_state_sibling = doubly_rnn_cell.get_zero_state('init_state_sibling')
        output, state_sibling = doubly_rnn_cell(initial_state_parent, initial_state_sibling, reuse=False)
        outputs[0], states_parent[0] = output, state_sibling

        for parent_idx, child_idxs in tree_idxs.items():
            state_parent = states_parent[parent_idx]
            state_sibling = initial_state_sibling
            for child_idx in child_idxs:
                output, state_sibling = doubly_rnn_cell(state_parent, state_sibling)
                outputs[child_idx], states_parent[child_idx] = output, state_sibling

    return outputs

## stick break

In [26]:
def hierarchical_sbp(tree_sticks_topic, tree_sticks_branch):
    tree_prob_topic = {}
    rest_topics = {}

    # calculate topic probability and save
    stick_topic = tree_sticks_topic[0]
    tree_prob_topic[0] = stick_topic
    rest_topics[0] = 1.-stick_topic
    for parent_idx, child_idxs in tree_idxs.items():
        rest_topic = rest_topics[parent_idx]
        rest_branch = 1.
        for child_idx in child_idxs:
            # calculate topic probability
            if child_idx == child_idxs[-1]: # last child
                prob_branch = rest_branch # phi
            else:
                stick_branch = tree_sticks_branch[child_idx] # psi
                prob_branch = stick_branch * rest_branch # phi

            if not child_idx in tree_idxs: # leaf childs
                prob_topic = prob_branch * rest_topic # pi
            else:
                stick_topic = tree_sticks_topic[child_idx] # upsilon
                prob_topic = stick_topic * prob_branch * rest_topic # pi

            # save topic probability and update rest stick length
            tree_prob_topic[child_idx] = prob_topic
            rest_branch = (1.- stick_branch) * rest_branch
            rest_topics[child_idx] = (1.-stick_topic)*prob_branch * rest_topic
            
    return tree_prob_topic

## build model

In [27]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_bow')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors
    prob_layer = lambda h: tf.nn.sigmoid(tf.matmul(latents_bow, h, transpose_b=True))
    
    tree_sticks_topic = doubly_rnn(config.dim_latent_bow, tree_idxs, output_layer=prob_layer, name='sticks_topic')
    tree_sticks_branch = doubly_rnn(config.dim_latent_bow, tree_idxs, output_layer=prob_layer, name='sticks_branch')

    tree_prob_topic = hierarchical_sbp(tree_sticks_topic, tree_sticks_branch)
    prob_topic = tf.concat(list(tree_prob_topic.values()), 1)

# decode bow
# with tf.variable_scope('shared', reuse=False):
#     embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

# bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    tree_hidden_topic_emb = doubly_rnn(config.dim_emb, tree_idxs, name='hidden_topic_emb')
    hidden_topic_emb = tf.concat(list(tree_hidden_topic_emb.values()), 0)
    
    means_topic_emb = tf.layers.Dense(units=config.dim_latent_topic, name='mean_topic_emb')(hidden_topic_emb)
    logvars_topic_emb = tf.layers.Dense(units=config.dim_latent_topic, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic_emb')(hidden_topic_emb)
    latents_topic_emb = sample_latents(means_topic_emb, logvars_topic_emb) # sample latent vectors
#     topic_embeddings = tf.layers.Dense(units=config.dim_emb, name='topic_emb')(latents_topic_emb)
    
#     topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
#     logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution
    
    topic_bow = tf.layers.Dense(units=config.dim_bow, activation=tf.nn.softmax, name='topic_emb')(latents_topic_emb)
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

## define loss

In [28]:
def get_tree_mask_reg(tree_idxs):
    tree_mask_reg = np.ones([len(topic_idxs), len(topic_idxs)], dtype=np.float32)
    parent_to_descendant_idxs = {parent_idx: get_descendant_idxs(parent_idx) for parent_idx in tree_idxs}
    
    for parent_idx, descendant_idxs in parent_to_descendant_idxs.items():
        for descendant_idx in descendant_idxs:
            tree_mask_reg[topic_idxs.index(parent_idx), topic_idxs.index(descendant_idx)] = tree_mask_reg[topic_idxs.index(descendant_idx), topic_idxs.index(parent_idx)] = 0.
            
    return tree_mask_reg

def get_descendant_idxs(parent_idx, descendant_idxs = None):
    if descendant_idxs is None: descendant_idxs = []
    
    child_idxs = tree_idxs[parent_idx]
    descendant_idxs += child_idxs
    for child_idx in child_idxs:
        if child_idx in tree_idxs: get_descendant_idxs(child_idx, descendant_idxs)
    return descendant_idxs

In [29]:
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_losses_kl_bow = compute_kl_losses(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std
topic_loss_kl_bow = tf.reduce_mean(topic_losses_kl_bow, [0]) #mean of kl_losses over batches

means_topic_emb_prior = tf.concat([tf.expand_dims(latents_topic_emb[topic_idxs.index(child_to_parent_idxs[topic_idx])], 0) if topic_idx in child_to_parent_idxs else tf.zeros([1, config.dim_latent_topic], dtype=tf.float32) for topic_idx in topic_idxs], 0)
topic_losses_kl_emb = compute_kl_losses(means_topic_emb, logvars_topic_emb, means_topic_emb_prior)
topic_loss_kl_emb = tf.reduce_sum(topic_losses_kl_emb, [0]) #sum of kl_losses over topics

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
tree_mask_reg = get_tree_mask_reg(tree_idxs)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(len(topic_idxs))) * tree_mask_reg)

global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

loss = topic_loss_recon + beta*topic_loss_kl_bow + topic_loss_kl_emb + config.reg * topic_loss_reg

In [30]:
# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)

grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]
opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

In [31]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl_bow, topic_loss_kl_emb, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_bow_mean, topic_loss_kl_emb_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_bow_mean, topic_loss_kl_emb_mean, topic_loss_reg_mean, ppl_mean

def print_topic_sample():
    topics_freq_bow_idxs = bow_idxs[sess.run(topics_freq_bow_indices)]
    assert len(topics_freq_bow_idxs) == len(topic_idxs)
    for topic_idx, topic_freq_bow_idxs in zip(topic_idxs, topics_freq_bow_idxs):
        print(topic_idx, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))

In [34]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','','TRAIN:','TM','','','', '','VALID:','TM','','','', ''],
                            ['Time','Ep','Ct','Beta','LOSS','PPL','NLL','KL(BOW)','KL(EMB)','REG','LOSS','PPL','NLL','KL(BOW)','KL(EMB)','REG']]))))

In [35]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl_bow, topic_loss_kl_emb, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_bow_train, topic_loss_kl_emb_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_bow_dev, topic_loss_kl_emb_dev, topic_loss_reg_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log, beta_eval = sess.run([tf.train.get_global_step(), beta])
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct,  '%.3f'%beta_eval, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_bow_train, '%.2f'%topic_loss_kl_emb_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_bow_dev, '%.2f'%topic_loss_kl_emb_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            
            # visualize topic
            print_topic_sample()

            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                                \
       Time  Ep    Ct   Beta    LOSS   PPL     NLL KL(BOW) KL(EMB)   REG   
1         2   0     0  0.000  394.60  1042  128.07    0.55  265.87  0.66   
1001     27   0  1000  0.176  428.68   630  115.33    1.41  312.69  0.58   
2001     27   0  2000  0.352  390.59   604  114.34    0.81  275.63  0.55   
2276      8   1     0  0.400  385.65   601  114.18    0.73  270.86  0.55   
3276     27   1  1000  0.576  373.81   591  113.85    0.53  259.38  0.52   
4276     27   1  2000  0.752  358.31   587  113.78    0.42  243.99  0.49   
4551      7   2     0  0.800  354.67   586  113.76    0.40  240.38  0.48   
5551     26   2  1000  0.976  344.19   583  113.66    0.33  230.04  0.45   
6551     27   2  2000  1.000  336.95   581  113.62    0.28  222.87  0.43   
6826      8   3     0  1.000  335.32   581  113.60    0.27  221.25  0.42   
7826     27   3  1000  1.000  330.39   579  113.63    0.24  216.32  0.41   
8826     26   3  2000  1.000  326.50   578  113.55    0.21  212.53  0.39   
9101      8   4     0  1.000  325.58   578  113.51    0.20  211.65  0.39   
10101    26   4  1000  1.000  322.61   576  113.47    0.18  208.73  0.38   
11101    26   4  2000  1.000  320.18   576  113.46    0.17  206.33  0.37   
11376     7   5     0  0.000  319.60   575  113.45    0.16  205.76  0.37   
12376    26   5  1000  0.176  317.63   574  113.41    0.19  203.84  0.36   
13376    27   5  2000  0.352  316.00   574  113.39    0.20  202.23  0.35   
13651     8   6     0  0.400  315.61   573  113.38    0.20  201.84  0.35   
14651    27   6  1000  0.576  314.27   573  113.40    0.19  200.50  0.34   
15651    27   6  2000  0.752  313.02   572  113.36    0.18  199.29  0.34   
15926     7   7     0  0.800  312.71   572  113.34    0.18  198.99  0.34   
16926    27   7  1000  0.976  311.70   571  113.32    0.17  198.01  0.33   
17926    27   7  2000  1.000  310.79   571  113.32    0.16  197.11  0.33   
18201     7   8     0  1.000  310.55   571  113.31    0.16  196.88  0.32   
19201    26   8  1000  1.000  309.73   571  113.30    0.15  196.09  0.32   
20201    27   8  2000  1.000  309.01   570  113.28    0.15  195.38  0.32   
20476     8   9     0  1.000  308.81   570  113.29    0.14  195.18  0.32   
21476    27   9  1000  1.000  308.22   570  113.30    0.14  194.58  0.31   
22476    27   9  2000  1.000  307.62   570  113.28    0.13  194.00  0.31   
...     ...  ..   ...    ...     ...   ...     ...     ...     ...   ...   
91001     7  40     0  0.000  297.71   561  113.02    0.10  184.39  0.26   
92001    25  40  1000  0.176  297.66   561  113.01    0.10  184.36  0.26   
93001    26  40  2000  0.352  297.62   561  113.02    0.10  184.32  0.26   
93276     7  41     0  0.400  297.61   561  113.02    0.10  184.31  0.26   
94276    27  41  1000  0.576  297.58   561  113.01    0.10  184.27  0.26   
95276    27  41  2000  0.752  297.55   561  113.01    0.10  184.24  0.26   
95551     7  42     0  0.800  297.54   561  113.01    0.10  184.23  0.26   
96551    27  42  1000  0.976  297.50   561  113.01    0.10  184.20  0.26   
97551    25  42  2000  1.000  297.47   561  113.01    0.10  184.17  0.26   
97826     7  43     0  1.000  297.46   561  113.01    0.10  184.16  0.26   
98826    26  43  1000  1.000  297.42   561  113.01    0.10  184.12  0.26   
99826    26  43  2000  1.000  297.39   561  113.01    0.10  184.09  0.26   
100101    7  44     0  1.000  297.38   561  113.01    0.10  184.08  0.26   
101101   27  44  1000  1.000  297.35   561  113.01    0.10  184.05  0.26   
102101   26  44  2000  1.000  297.31   561  113.00    0.10  184.02  0.26   
102376    7  45     0  0.000  297.30   561  113.01    0.10  184.01  0.26   
103376   25  45  1000  0.176  297.28   561  113.01    0.10  183.98  0.26   
104376   25  45  2000  0.352  297.24   560  113.00    0.10  183.95  0.26   
104651    7  46     0  0.400  297.23   560  113.00    0.10  183.94  0.26   
105651   26  46  1000  0.576  297.21   560  113.00    0.10  183.91  0.26   
10

0  BOW: ! bought nice quality 'm - cover inside pockets pro
1  BOW: ! cover perfectly & carry pockets nice color room inside
2  BOW: nice bought sleeve ; perfect quality 'm pockets room back
3  BOW: color bought nice ! quality perfectly love 'm carry pro
10  BOW: ! perfect bought price perfectly pockets back room ; 'm
11  BOW: carry sleeve bought ; cover quality price room put &
20  BOW: bought & recommend carry nice perfect quality love price small
21  BOW: nice sleeve bought carry perfectly ! pockets room made ...
30  BOW: love sleeve bought ; perfect quality 'm pro pockets inside
31  BOW: color cover quality bought recommend sleeve perfect pockets ! back
100  BOW: & cover perfect nice ! color love pockets pro 'm
101  BOW: carry quality bought sleeve cover perfectly ; pockets - put
110  BOW: love recommend carry color quality ; cover nice perfectly pro
111  BOW: & sleeve cover love price room pockets perfectly put back
200  BOW: perfectly quality sleeve color love nice recommend pro 

# confirm variables

In [21]:
debug_value([latents_topic_emb[:, :10]])

strided_slice_9:0 : [[-0.27247018 -0.17721193  0.09622851 -0.3813508  -0.56140953 -0.328331
   0.9186956   0.256747   -0.3996384   0.40822035]
 [-1.1523937  -0.20411795  0.12704408 -1.0915084   0.13742964  0.40063304
   0.6096825  -0.73957384 -0.524794   -0.48537108]
 [-0.24014454  0.16527298 -0.5442626  -0.8756025   0.38910925  0.64856243
  -0.98533165 -0.49310398  0.20743468  0.60926753]
 [-0.14281085 -0.00473346  0.3963806  -0.60402584  1.1851643   0.64240825
  -0.56131923  0.08793946  0.8663161   0.12775922]
 [-0.58583164  1.7180016  -1.1750451  -0.7574285  -0.69132227  0.09512067
   2.3391447   1.0156784  -0.01139697 -1.0669242 ]
 [ 0.17385426 -1.000231   -0.21883528  1.2355777  -1.0963326  -0.16964464
   0.87885475  0.01161904 -1.059508   -0.20077345]
 [ 0.36295784 -0.6584745   0.66140836  0.45562926  1.1263165   1.3476317
   2.2007372   0.59743464 -1.2357106  -1.0083864 ]
 [ 0.44753993 -1.503215    1.3551549  -0.8612783  -0.10684672  0.4670496
   0.7363037   0.51386774  0.280191

In [23]:
debug_value([topic_embeddings[:, :10]])

strided_slice_11:0 : [[-0.06089201  0.08843822  0.02681581  0.04435141  0.22810166 -0.08150685
   0.09354918 -0.05975756  0.09338889  0.15735947]
 [-0.04057656  0.02809375 -0.0744636  -0.08872357  0.5012574  -0.13109821
   0.11898467 -0.1113497  -0.20294835 -0.04296541]
 [-0.10577152  0.0261583  -0.09760163  0.03949188 -0.00745398  0.01887681
  -0.05485298 -0.18335932  0.04143422  0.17509793]
 [-0.00470228 -0.05082985  0.04451936  0.07047128  0.37365606 -0.09635599
   0.12019593 -0.01452139 -0.02394241  0.1091679 ]
 [-0.08709863  0.10658363 -0.20022729  0.06692281  0.34322727  0.06838685
  -0.05020845  0.28062564  0.1415717  -0.00208222]
 [ 0.079356   -0.03839519  0.14164594  0.0605994   0.2156288  -0.17053501
   0.19101676 -0.29390422 -0.29393244  0.23207906]
 [-0.2002696   0.19944216  0.07815433  0.11582892  0.08108386 -0.16934088
  -0.19562826  0.04315841 -0.08215998  0.16240826]
 [ 0.07134811  0.3490946   0.07403649 -0.01986381  0.1337995  -0.08985814
  -0.06378951  0.08622152  0.1

In [22]:
debug_value([prob_topic[0]])

strided_slice_11:0 : [0.5031053  0.10442396 0.05414205 0.05484921 0.07081123 0.07056253
 0.03681501 0.03763846 0.03353582 0.03411642]


In [23]:
debug_value([prob_topic[3]])

strided_slice_12:0 : [0.4666806  0.14801824 0.07320257 0.0707541  0.05691184 0.06307651
 0.02904052 0.03196352 0.02882889 0.03152321]


In [ ]:
debug_value([tf.exp(-tf.divide(topic_losses_recon, n_bow))])

### test

In [12]:
debug_shape([bow, hidden_bow, latents_bow, prob_topic, bow_embeddings, topic_embeddings, topic_bow, prob_bow])

Placeholder:0 : (32, 4022)
topic/enc/dropout/cond/Merge:0 : (32, 256)
topic/enc/add:0 : (32, 32)
topic/enc/prob/Softmax:0 : (32, 50)
embedding_lookup:0 : (4022, 256)
topic/dec/topic_emb:0 : (50, 256)
topic/dec/Softmax:0 : (50, 4022)
topic/dec/Log:0 : (32, 4022)


In [27]:
debug_shape([topic_losses_recon, topic_loss_recon, n_bow, ppls, topic_embeddings_norm, tf.expand_dims(topic_angles_mean, -1), topic_angles_vars])

Sum:0 : (32,)
Neg:0 : ()
Sum_2:0 : (32,)
Neg_1:0 : (32,)
truediv_1:0 : (50, 256)
ExpandDims_1:0 : (1,)
Mean_3:0 : ()


In [14]:
debug_value([tf.reduce_sum(tf.square(topic_embeddings_norm), 1)], return_value=True)[0]

array([1.        , 0.99999994, 0.9999999 , 1.        , 1.        ,
       1.0000001 , 0.9999999 , 1.0000001 , 1.0000001 , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.9999999 , 0.9999999 , 0.99999994, 1.        ,
       1.        , 0.9999999 , 1.0000001 , 1.        , 1.        ,
       1.        , 0.99999994, 1.        , 0.99999994, 0.99999994,
       1.0000001 , 0.9999999 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.9999999 , 1.        ,
       0.9999999 , 1.        , 1.        , 1.        , 0.99999994,
       1.0000001 , 1.        , 1.        , 0.99999994, 1.        ],
      dtype=float32)

In [15]:
debug_value([tf.reduce_sum(prob_topic, -1), tf.reduce_sum(topic_bow, -1), tf.reduce_sum(tf.exp(prob_bow), 1)])

Sum_4:0 : [1.0000001  1.         1.0000001  0.99999994 1.         1.
 1.         0.99999994 0.99999994 1.         1.         1.
 0.99999994 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.9999999
 1.         0.99999994 1.         1.0000001  0.99999994 1.
 1.         1.        ]
Sum_5:0 : [1.         0.99999994 1.         0.99999994 0.99999994 1.
 1.         0.9999998  1.         1.         1.         1.
 1.         1.         1.         1.0000001  0.99999994 1.
 1.         0.99999994 0.9999999  0.99999994 1.0000001  1.
 1.         1.         1.         0.99999994 0.9999999  1.
 1.         1.         0.99999994 1.         0.99999994 1.
 1.         0.9999999  1.         1.         1.         1.
 0.99999994 1.         0.99999994 0.99999994 0.99999994 0.99999994
 1.         1.        ]
Sum_6:0 : [1.         1.         1.0000001  0.99999994 1.         1.
 0.9999999  0.99999994 0.9999999  1.         1.0000001  1.
 0.9999999  1.        

In [16]:
sigma_bow = tf.exp(0.5 * logvars_bow)
dist_bow = tfd.Normal(means_bow, sigma_bow)
dist_std = tfd.Normal(0., 1.)
topic_loss_kl_tmp = tf.reduce_mean(tf.reduce_sum(tfd.kl_divergence(dist_bow, dist_std), 1))

In [17]:
debug_value([topic_loss_recon, topic_loss_kl, topic_loss_kl_tmp])

Neg:0 : 405.38312
Mean_1:0 : 0.32056683
Mean_4:0 : 0.32056683


In [110]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [111]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

((32, 32), (32, 32), (32,), (32, 32))

In [112]:
_output_logits

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [109]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


NameError: name 'dec_target_idxs_do' is not defined

In [44]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

(120, 46)

In [31]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

((120, 46, 20000), (120, 46), (120, 46))

In [32]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [33]:
_idxs = _dec_target_idxs_do

In [35]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [36]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

9.903732

In [37]:
_recon_loss

9.903732

In [38]:
_kl_losses.shape

(120,)